In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

In [ ]:
# Lade Daten
data = pd.read_csv('/Users/robinfischer/Desktop/pyfin_assig_data.csv')


In [ ]:
data['form_rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change, periods=6, fill_method=None)

data['hold_rets'] = data.groupby('permno')['price'].transform(pd.Series.pct_change, periods=1, fill_method=None).shift(-2)


In [ ]:
# Construct indicator for a 'good' stock
data['good_stock'] = 0
data.loc[~data['form_rets'].isnull(), 'good_stock']=1


In [ ]:
# Assign stocks into 10 equal groups based on formation period returns
data['port'] = data.loc[data['good_stock']==1].groupby('date')['form_rets'].transform(pd.qcut, q=10, labels=range(1,11)).astype(str)


In [ ]:
port_df = data.loc[~data['port'].isnull()].groupby(['date','port'])['hold_rets'].mean().reset_index()

port_df = port_df.pivot(index = 'date', columns = 'port', values = 'hold_rets').reset_index()


In [ ]:
# Calculate the returns
port_df['Mom_rets'] = port_df['10']-port_df['1']

In [ ]:
#Visuals
port_df['Mom_cum_rets'] = (1+port_df['Mom_rets']).cumprod()
px.line(port_df, y='Mom_cum_rets', x='date', labels={'Mom_cum_rets': 'Cumulative returns'}, title = 'Momentum Strategy').show()
